# Purpose of this project

* ## find a correlaion between the drama script and content completion rate

---
---
---

https://www.whats-on-netflix.com/news/are-completion-rates-key-to-netflix-cancelations/
<img src="./image/1.PNG" width="700" />
<img src="./image/3.PNG" width="700" />

---
---
---

# Can we measure the completion rate through the script?

## 1. Gather two script data 
* The Irregulars : the lowest completion rate in upper image
* Squid Game : the highest completion rate in upper image > This is what I'm going to present 
 

## 2. Get dialogues without actions
* dropout actions : measure the depth between the characters using network analysis, not mise en scene(in the scene)
* make dataframe : make a dataframe with charector and dialogue

 

## 3. Measure the depth of emotions 
* run a network analysis : Using the dataframe, do a network analysis with each of episode of two drama-The Irregulars and Squid Game
* Compare the firues 

---
---
---

## Step 1. Gather two script data 
* The Irregulars : the lowest completion rate in upper image
* Squid Game : the highest completion rate in upper image > This is what I'm going to present 

In [1]:
# install used pakages 
# pip install selenium

In [2]:
# import modules I'm going to use

import selenium
import numpy as np
import pandas as pd

# for automation in further data gathering, I used selenium to crawl the script data 
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Here is a useful link about the usage of selenium!
# https://jackerlab.com/python-selenium/
# It's a kind of dynamic crawler that help to gather the data after some kind of action, 
# ex)you've loged in or scroll down 

In [3]:
def dynamic_web_crawler(url, xpath):
    """
    function name : dynamic_web_crawler
    input parameter : url, xpath 
    output parameter : crawled data
    """

    # to avoid unexpected exception, I used try-except-finally 
    try:
        # go to web page
        driver.get(url)

        # wait until the script page id loaded 
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.XPATH, xpath)))

        # find the element by xpath in html
        # which contains the script data
        elements = driver.find_elements(By.XPATH, xpath)
        # extract the text data using list comprehension
        data = [element.text for element in elements]
        # if all done, return the text data 
        return data
    except:
        # if error comes up while running the code, shut down the driver
        print("error occures")
        driver.quit()
    finally:
        # if all done normally, shut down the driver 
        driver.quit()
        

In [4]:
# Before calling the function, defind the input parameters 
url = 'https://scrapsfromtheloft.com/tv-series/squid-game-s01e09-one-lucky-day-transcript/'
# script data location in html
xpath = '//*[@id="main"]/div/section/div/div/div/div[7]/div'

In [5]:
# define the webdriver as driver to open Chrome browser as a global variable
driver = webdriver.Chrome()

In [6]:
# call the function I made
script = dynamic_web_crawler(url, xpath)

In [7]:
# check the return value of the function 
print("type of script variable is", type(script), '\n')
print("length of script variable is", len(script), '\n')
print("contents of script variable is \n", script)

type of script variable is <class 'list'> 

length of script variable is 1 

contents of script variable is 
 ['Original release date: September 17, 2021\nThe sixth and final game is the Squid Game, with Gi-hun on offense and Sang-woo on defense. Gi-hun beats Sang-woo, but refuses to complete the game. He invokes the third clause to end the game, but Sang-woo stabs himself and asks Gi-hun to use the prize money to help Sang-woo’s mother. Gi-hun returns home to find his mother already dead. A year later, Gi-hun is still traumatized by his experience and has barely touched his prize money. One night he receives a gold game invitation card, and in answering it finds Player 001, Oh Il-nam, still alive but dying. Il-nam is one of the VIPs, the “Host” (who co-ran the game alongside the Front Man, In-ho), and the mastermind behind the game operation, which was made to entertain the bored super rich like himself. Il-nam explains that both the poverty-stricken and uber-wealthy lead monotonous l

## Step 2. Get dialogues without actions
* dropout actions : measure the depth between the characters using network analysis, not mise en scene(in the scene)
* make dataframe : make a dataframe with charector and dialogue

## make a function named 'pp1'(preprocessing1)
* delete the text that matches the condition
    * first line of the script : release date
    * second paragraph of the script : summary of the episode 
    * the actual script starts with the three star marks (* * *)


<img src="./image/4.PNG" width="700" />

In [8]:
def pp1(script) :  
    """
    function name : pp1
    input parameter : script
    output parameter : preprocessed script(erase the text which is not script)
    """

    full_script = script[0]  
    # second paragraph of the script : summary of the episode 
    full_script = full_script.split('\n', 2)[2]
    # the actual script starts with the three star marks (* * *)
    full_script = full_script.split('*', 3)[3]
    return full_script

In [9]:
# call pp1
pp1_script = pp1(script)

In [10]:
# result of pp1 
pp1_script 

'\n[dramatic music]\n[dramatic music continues]\n[masked manager] Player 218 and Player 456, welcome to the final game. Before we begin, we’ll flip a coin to decide who plays offense or defense. Please choose a shape. Triangle or square.\nTriangle.\n[coin clangs, whooshes]\n[masked manager] It’s triangle. Please choose either offense or defense.\n[inhales sharply]\nOffense.\n[masked manager] Player 456 will play offense. Player 218 will defend.\n[door clangs]\n[masked manager] For the final game, you’ll both be playing Squid Game.\n[VIP 1] Squid Game.\nWhat an odd name.\n[Front Man] It was originally a children’s game that was played in Korea many years ago.\n[VIP 1] Hmm.\n[masked manager] Let me explain the rules of Squid Game.\nOne, the attacker must enter the squid shape court, run past the defense, and then tap the area inside the squid’s head with his foot to secure the win.\nTwo, the defender must block the attacker’s advance and force them outside of the court’s bounds in order 

## make a function named 'pp2'(preprocessing2)
* dropout the actions in the script
    * split the text data with the newline delimeter('\n[')
    * if there is just a bracelet(ex. []) with no other line, then that's an Action(in korean 지문)
    * if there is a bracelet with some lines, then that is a dialogue(in korean 대사)
        * in this case, the words in bracelet is the character's name 

<img src="./image/5.jpg" width="700" />

In [11]:
def isDialogue(line) : 
    """
    function name : isDialogue
    input parameter : line
    output parameter : Boolean
    """
    bracelet_idx = line.find(']')
    line_len = len(line)
    print(line, bracelet_idx, line_len )
    # check if line has no words
    if (line_len < 1) |\
        ((bracelet_idx>0) & ((bracelet_idx+1)==line_len)):
        #check if there is just action with braclet 
        # then, it is an action so return False 
        return False 
    else :  
        return True

In [12]:
def pp2(script) :  
    """
    function name : pp2
    input parameter : script
    output parameter : preprocessed script(dropout Actions)
    """
    # split the text data with the newline delimeter('\n[')
    # I used the delimeter '\n[' because there is newline('\n') between some dialogues 
    full_script = script.split('\n[')
    # using list comprehension, dropout actions, gather dialogues
    full_script = [line for line in full_script if isDialogue(line)]
    return full_script

In [13]:
# pp2(pp1_script)
pp2_script = pp2(pp1_script)

 -1 0
dramatic music] 14 15
dramatic music continues] 24 25
masked manager] Player 218 and Player 456, welcome to the final game. Before we begin, we’ll flip a coin to decide who plays offense or defense. Please choose a shape. Triangle or square.
Triangle. 14 197
coin clangs, whooshes] 21 22
masked manager] It’s triangle. Please choose either offense or defense. 14 71
inhales sharply]
Offense. 15 25
masked manager] Player 456 will play offense. Player 218 will defend. 14 69
door clangs] 11 12
masked manager] For the final game, you’ll both be playing Squid Game. 14 70
VIP 1] Squid Game.
What an odd name. 5 36
Front Man] It was originally a children’s game that was played in Korea many years ago. 9 87
VIP 1] Hmm. 5 11
masked manager] Let me explain the rules of Squid Game.
One, the attacker must enter the squid shape court, run past the defense, and then tap the area inside the squid’s head with his foot to secure the win.
Two, the defender must block the attacker’s advance and force t

In [14]:
pp2_script

['masked manager] Player 218 and Player 456, welcome to the final game. Before we begin, we’ll flip a coin to decide who plays offense or defense. Please choose a shape. Triangle or square.\nTriangle.',
 'masked manager] It’s triangle. Please choose either offense or defense.',
 'inhales sharply]\nOffense.',
 'masked manager] Player 456 will play offense. Player 218 will defend.',
 'masked manager] For the final game, you’ll both be playing Squid Game.',
 'VIP 1] Squid Game.\nWhat an odd name.',
 'Front Man] It was originally a children’s game that was played in Korea many years ago.',
 'VIP 1] Hmm.',
 'masked manager] Let me explain the rules of Squid Game.\nOne, the attacker must enter the squid shape court, run past the defense, and then tap the area inside the squid’s head with his foot to secure the win.\nTwo, the defender must block the attacker’s advance and force them outside of the court’s bounds in order to win.\nThree, if a situation arises, in which either player is unable 

## make a function named 'pp3'(preprocessing3)

* I'll use the function name 'pp3'(preprocessing3)
    * split the text data with the newline delimeter(']')
    * put this data in DataFrame (columns = ['character', 'dialogue'])

In [15]:
def pp3(script) :
    """
    function name : pp3
    input parameter : script
    output parameter : dataframe of script
    """    
    # using split methods, devide the line into character and diagloue
    # and make it as an pandas dataframe 
    script_df = pd.DataFrame({  'character':[line.split(']')[0].strip() for line in script]
                              , 'dialogue':[line.split(']')[1].lstrip().replace('\n', '') for line in script]})
    return script_df

In [16]:
pp3_script = pp3(pp2_script)

In [17]:
pp3_script.head(10)

,character,dialogue
0,masked manager,"Player 218 and Player 456, welcome to the fina..."
1,masked manager,It’s triangle. Please choose either offense or...
2,inhales sharply,Offense.
3,masked manager,Player 456 will play offense. Player 218 will ...
4,masked manager,"For the final game, you’ll both be playing Squ..."
5,VIP 1,Squid Game.What an odd name.
6,Front Man,It was originally a children’s game that was p...
7,VIP 1,Hmm.
8,masked manager,"Let me explain the rules of Squid Game.One, th..."
9,masked manager,That would be referring to the player’s death....


## make a function named 'modify_dialogue'(modify_dialogue)
* There are some errors in the dataFrame
    * some line don't follow the rules 
    * find the irregular one and modify like below 

<img src="./image/6.jpg" width="500" />

In [18]:
#check the dataframe column 'character' which does not have a character's name 
pp3_script['character'].value_counts().index.to_list()

['Gi-hun',
 'Front Man',
 'masked manager',
 'breathes deeply',
 'Il-nam',
 'Sang-woo’s mom',
 'gasps',
 'grunts',
 'VIP 1',
 'sighs',
 'in Korean',
 'Sang-woo',
 'inhales',
 'chuckles',
 'pensive music continues',
 'woman',
 'VIP 3',
 'inhales sharply',
 'in English',
 'shaky breathing',
 'inhales deeply',
 'exhales',
 'female reporter',
 'somber music',
 'woman on PA',
 'VIP 2',
 'Il-nam breathing heavily',
 'man groans',
 'bank manager',
 'clicks tongue',
 'tense music',
 'Il-nam exhales',
 'upbeat music on TV',
 'hairdresser',
 'Il-nam gulping water',
 'Gi-hun gasps',
 'ticking',
 'shouts angrily',
 'Il-nam grunting',
 'clock ticking',
 'dramatic music continues',
 'man',
 'heart rate monitor beeping',
 'bank manager chuckles',
 'Gi-hun sighs',
 'labored breathing',
 'sucks teeth, inhales',
 'Gi-hun breathing heavily',
 'nervous chuckle',
 'whimpering',
 'rain pouring',
 'yelps',
 'dramatic music',
 'groans',
 'Sang-woo cries in pain',
 'VIP 1, in English',
 'dramatic music swells'

In [19]:
# extract some actions in character column
notcharectors_list = [
 'breathes deeply',
 'gasps',
 'grunts',
 'sighs',
 'in Korean',
 'inhales',
 'chuckles',
 'pensive music continues',
 'inhales sharply',
 'in English',
 'shaky breathing',
 'inhales deeply',
 'exhales',
 'somber music',
 'Il-nam breathing heavily',
 'man groans',
 'clicks tongue',
 'tense music',
 'Il-nam exhales',
 'upbeat music on TV',
 'Il-nam gulping water',
 'Gi-hun gasps',
 'ticking',
 'shouts angrily',
 'Il-nam grunting',
 'clock ticking',
 'dramatic music continues',
 'heart rate monitor beeping',
 'bank manager chuckles',
 'Gi-hun sighs',
 'labored breathing',
 'sucks teeth, inhales',
 'Gi-hun breathing heavily',
 'nervous chuckle',
 'whimpering',
 'rain pouring',
 'yelps',
 'dramatic music',
 'groans',
 'Sang-woo cries in pain',
 'dramatic music swells',
 'heavy breath',
 'pants',
 'somber music continues',
 'Sang-woo gags',
 'Gi-hun whimpers',
 'Gi-hun crying',
 'gulps',
 'wailing',
 'chokes',
 'evangelist',
 'suspenseful music',
 'heavy breathing',
 'automated voice',
 'beeps',
 'beeping',
 'panting',
 'pensive music',
]


# join the noncharacters_list with join function 
nochar_str = '|'.join(notcharectors_list)

In [20]:
# using the condition I made in upper cell(nonchar_str), get the rows to be modifed 
redo_index = pp3_script[pp3_script['character'].str.contains(nochar_str)].index

In [21]:
redo_index

Int64Index([  2,  11,  14,  15,  16,  17,  19,  22,  23,  24,  25,  26,  28,
             29,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  44,  45,
             49,  50,  51,  52,  53,  54,  55,  56,  58,  59,  60,  61,  62,
             64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  78,  79,
             80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
             93,  95,  98,  99, 103, 104, 105, 107, 109, 113, 115, 117, 121,
            123, 125, 128, 130],
           dtype='int64')

In [22]:
def modify_dialogue(line):
    """
    function name : modify_dialogue
    input parameter : line
    output parameter : None 
    """    
    print("!")
    # get the index of the parameter
    append_tar = line.index
    # using + operator, concat the problem line with the line before the problem line
    for i in append_tar:
        pp3_script.loc[i-1, 'dialogue'] = pp3_script.loc[i-1, 'dialogue'] + pp3_script.loc[i, 'dialogue']
        


In [23]:
# call modify_dialogue function
modify_dialogue(pp3_script[pp3_script['character'].str.contains(nochar_str)])

!


In [24]:
# pd.options.display.max_colwidth = 2000
# check the result 
pp3_script['dialogue'].head(11)

0     Player 218 and Player 456, welcome to the fina...
1     It’s triangle. Please choose either offense or...
2                                              Offense.
3     Player 456 will play offense. Player 218 will ...
4     For the final game, you’ll both be playing Squ...
5                          Squid Game.What an odd name.
6     It was originally a children’s game that was p...
7                                                  Hmm.
8     Let me explain the rules of Squid Game.One, th...
9     That would be referring to the player’s death....
10    Among all the games that kids played back then...
Name: dialogue, dtype: object

In [25]:
pp3_script

,character,dialogue
0,masked manager,"Player 218 and Player 456, welcome to the fina..."
1,masked manager,It’s triangle. Please choose either offense or...
2,inhales sharply,Offense.
3,masked manager,Player 456 will play offense. Player 218 will ...
4,masked manager,"For the final game, you’ll both be playing Squ..."
...,...,...
128,breathes deeply,"…who you people are, and how you can do these ..."
129,Front Man,Player 456.Don’t get any absurd ideas.
130,sighs,Don’t get any absurd ideas.
131,Gi-hun,It wasn’t a dream. I can’t forgive you… for ev...


In [26]:
# remove the action line in the dataframe
pp3_script.drop(redo_index, inplace=True)
pp3_script.reset_index(inplace=True, drop=True)

In [27]:
# check the result 
pp3_script.head(11)

,character,dialogue
0,masked manager,"Player 218 and Player 456, welcome to the fina..."
1,masked manager,It’s triangle. Please choose either offense or...
2,masked manager,Player 456 will play offense. Player 218 will ...
3,masked manager,"For the final game, you’ll both be playing Squ..."
4,VIP 1,Squid Game.What an odd name.
5,Front Man,It was originally a children’s game that was p...
6,VIP 1,Hmm.
7,masked manager,"Let me explain the rules of Squid Game.One, th..."
8,masked manager,That would be referring to the player’s death....
9,Front Man,Among all the games that kids played back then...
